In [1]:
import pandas as pd

In [10]:
df = pd.read_csv('../outnow_2.csv')

In [11]:
df.head()

,Unnamed: 0,title,keywords,affiliation_id,cited_by_count,AGRI,ARTS,BIOC,BUSI,CENG,...,MEDI,MULT,NEUR,NURS,PHAR,PHYS,PSYC,SOCI,VETE,title_keywords
0,0,Reduction of algal wall-growth biofilm for the...,Surfactant;Microalgae;Chlorococcum;Biofilm;Pho...,0.0,1.0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,Reduction of algal wall-growth biofilm for the...
1,1,Dietary anthocyanins inhibit insulin fibril fo...,Cytotoxicity;Cyanidin;Malvidin;Anthocyanins;In...,1.0,2.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Dietary anthocyanins inhibit insulin fibril fo...
2,2,Metformin delivery via iontophoresis based on ...,Transdermal drug delivery;κ-Carrageenan;Metformin,2.0,5.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Metformin delivery via iontophoresis based on ...
3,3,Experimental investigations of CO2 absorption ...,CO2 capture;CO2 absorption;Amine blends;Equili...,3.0,7.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Experimental investigations of CO2 absorption ...
4,4,High-throughput screening of highly active and...,Single-atom catalysts;Tungsten diboride;Electr...,4.0,1.0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,High-throughput screening of highly active and...


In [6]:
from gensim.models import Word2Vec
import numpy as np
import gensim
print(np.__version__)

1.24.3


In [12]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download the necessary NLTK data files (only needed once)
nltk.download('punkt')
nltk.download('stopwords')

# Function to tokenize a sentence and remove stop words
def preprocess_text(sentence):
    # Tokenize the sentence
    tokens = word_tokenize(sentence)
    
    # Convert tokens to lowercase
    tokens = [token.lower() for token in tokens]
    
    # Get the list of English stop words
    stop_words = set(stopwords.words('english'))
    
    # Remove stop words
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    return filtered_tokens

# Example usage
df['token'] = df['title_keywords'].apply(lambda x: preprocess_text(x))

[nltk_data] Downloading package punkt to /Users/ming/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ming/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
df['token']

0        [reduction, algal, wall-growth, biofilm, culti...
1        [dietary, anthocyanins, inhibit, insulin, fibr...
2        [metformin, delivery, via, iontophoresis, base...
3        [experimental, investigations, co2, absorption...
4        [high-throughput, screening, highly, active, s...
                               ...                        
18390    [guidelines, oie, (, world, organisation, anim...
18391     [accreditation, veterinary, inspection, systems]
18392                                        [remember, ?]
18393    [history, association, german, vete, rinary, o...
18394    [comparative, effect, selenium, wheat, ,, barl...
Name: token, Length: 18395, dtype: object

In [15]:


# Step 2: Train Word2Vec model
# Create a Word2Vec model using the tokenized column
w2v_model = Word2Vec(sentences=df['token'], vector_size=100, window=5, min_count=1, workers=4)

# Step 3: Convert words to vectors
def row_to_vector(row, model):
    vectors = []
    for word in row:
        if word in model.wv:
            vectors.append(model.wv[word])
    if vectors:
        # Average the vectors for the row
        return np.mean(vectors, axis=0)
    else:
        # Return a zero vector if no words were in the vocabulary
        return np.zeros(model.vector_size)

df['Vector'] = df['token'].apply(lambda x: row_to_vector(x, w2v_model))

# Step 4: Save or use the vectors
print(df['Vector'])


0        [-0.30878165, 0.1294211, 0.51457447, 0.5353811...
1        [-0.41651464, 0.30941254, 0.384592, 0.51573336...
2        [-0.46091264, 0.13170087, 0.37941128, 0.607616...
3        [-0.37255245, 0.124440625, 0.6665848, 0.790595...
4        [-0.6062769, 0.35366526, 0.4606807, 0.74103546...
                               ...                        
18390    [-0.17984593, 0.4254097, 0.04565878, 0.1793838...
18391    [-0.072829485, -0.10541179, 0.30108836, 0.3020...
18392    [0.14343226, 0.1805195, 0.34269145, 0.21067818...
18393    [-0.041794647, 0.087642215, 0.15884243, 0.1285...
18394    [-0.16870943, 0.41331536, 0.0097645875, 0.2740...
Name: Vector, Length: 18395, dtype: object


In [16]:
df.head()

,Unnamed: 0,title,keywords,affiliation_id,cited_by_count,AGRI,ARTS,BIOC,BUSI,CENG,...,NEUR,NURS,PHAR,PHYS,PSYC,SOCI,VETE,title_keywords,token,Vector
0,0,Reduction of algal wall-growth biofilm for the...,Surfactant;Microalgae;Chlorococcum;Biofilm;Pho...,0.0,1.0,0,0,1,0,1,...,0,0,0,0,0,0,0,Reduction of algal wall-growth biofilm for the...,"[reduction, algal, wall-growth, biofilm, culti...","[-0.30878165, 0.1294211, 0.51457447, 0.5353811..."
1,1,Dietary anthocyanins inhibit insulin fibril fo...,Cytotoxicity;Cyanidin;Malvidin;Anthocyanins;In...,1.0,2.0,0,0,1,0,0,...,0,0,0,0,0,0,0,Dietary anthocyanins inhibit insulin fibril fo...,"[dietary, anthocyanins, inhibit, insulin, fibr...","[-0.41651464, 0.30941254, 0.384592, 0.51573336..."
2,2,Metformin delivery via iontophoresis based on ...,Transdermal drug delivery;κ-Carrageenan;Metformin,2.0,5.0,0,0,1,0,0,...,0,0,0,0,0,0,0,Metformin delivery via iontophoresis based on ...,"[metformin, delivery, via, iontophoresis, base...","[-0.46091264, 0.13170087, 0.37941128, 0.607616..."
3,3,Experimental investigations of CO2 absorption ...,CO2 capture;CO2 absorption;Amine blends;Equili...,3.0,7.0,0,0,0,0,1,...,0,0,0,0,0,0,0,Experimental investigations of CO2 absorption ...,"[experimental, investigations, co2, absorption...","[-0.37255245, 0.124440625, 0.6665848, 0.790595..."
4,4,High-throughput screening of highly active and...,Single-atom catalysts;Tungsten diboride;Electr...,4.0,1.0,0,0,0,0,0,...,0,0,0,1,0,0,0,High-throughput screening of highly active and...,"[high-throughput, screening, highly, active, s...","[-0.6062769, 0.35366526, 0.4606807, 0.74103546..."


In [ ]:

vector_expanded = pd.DataFrame(df['Vector'].tolist(), index=df.index)
vector_expanded.columns = [f'Vector_{i}' for i in range(vector_expanded.shape[1])]
df_expanded = pd.concat([df.drop(columns=['Vector']), vector_expanded], axis=1)
df_expanded

,Unnamed: 0,title,keywords,affiliation_id,cited_by_count,AGRI,ARTS,BIOC,BUSI,CENG,...,Vector_90,Vector_91,Vector_92,Vector_93,Vector_94,Vector_95,Vector_96,Vector_97,Vector_98,Vector_99
0,0,Reduction of algal wall-growth biofilm for the...,Surfactant;Microalgae;Chlorococcum;Biofilm;Pho...,0.0,1.0,0,0,1,0,1,...,0.423304,0.109352,0.247982,0.041144,0.788401,0.601200,-0.101322,-0.685259,-0.027233,-0.107829
1,1,Dietary anthocyanins inhibit insulin fibril fo...,Cytotoxicity;Cyanidin;Malvidin;Anthocyanins;In...,1.0,2.0,0,0,1,0,0,...,0.456145,0.121678,0.302923,-0.084340,0.728554,0.461968,-0.003212,-0.506743,-0.001962,-0.019336
2,2,Metformin delivery via iontophoresis based on ...,Transdermal drug delivery;κ-Carrageenan;Metformin,2.0,5.0,0,0,1,0,0,...,0.508919,0.145826,0.290357,0.084980,0.760351,0.543809,-0.090790,-0.675838,-0.028711,-0.007196
3,3,Experimental investigations of CO2 absorption ...,CO2 capture;CO2 absorption;Amine blends;Equili...,3.0,7.0,0,0,0,0,1,...,0.386102,0.104899,0.145045,0.070964,0.966504,0.847344,-0.266861,-0.692086,0.021537,-0.316819
4,4,High-throughput screening of highly active and...,Single-atom catalysts;Tungsten diboride;Electr...,4.0,1.0,0,0,0,0,0,...,0.632707,0.056218,0.390430,0.008308,1.070381,0.863674,0.107317,-0.687583,-0.049549,-0.205274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18390,18390,Guidelines of the OIE (World organisation for ...,NaN,NaN,1.0,0,0,0,0,0,...,0.676916,0.023439,0.701723,0.050188,1.012036,0.494231,0.499359,-0.504574,-0.116716,-0.221784
18391,18391,Accreditation of veterinary inspection systems,NaN,NaN,1.0,0,0,0,0,0,...,0.385714,0.117316,0.250398,0.265082,0.601075,0.467043,-0.098883,-0.590041,-0.121477,-0.163304
18392,18392,Do you remember?,NaN,NaN,0.0,0,0,0,0,0,...,0.838214,0.145714,0.947118,0.128433,0.923074,0.442115,0.471448,-0.876138,-0.130669,-0.151427
18393,18393,The history of the Association of German Vete ...,NaN,NaN,1.0,0,0,0,0,0,...,0.376455,0.040270,0.372066,-0.009556,0.372351,0.270971,0.188838,-0.326120,-0.036026,-0.063192


In [18]:
df_expanded = df_expanded.drop(columns='Unnamed: 0')

In [20]:
df_expanded.to_csv("outnow_3")

In [25]:
df = pd.read_csv("../outnow_2.csv")

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18395 entries, 0 to 18394
Data columns (total 33 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      18395 non-null  int64  
 1   title           18395 non-null  object 
 2   keywords        16407 non-null  object 
 3   affiliation_id  16407 non-null  float64
 4   cited_by_count  18395 non-null  float64
 5   AGRI            18395 non-null  int64  
 6   ARTS            18395 non-null  int64  
 7   BIOC            18395 non-null  int64  
 8   BUSI            18395 non-null  int64  
 9   CENG            18395 non-null  int64  
 10  CHEM            18395 non-null  int64  
 11  COMP            18395 non-null  int64  
 12  DECI            18395 non-null  int64  
 13  DENT            18395 non-null  int64  
 14  EART            18395 non-null  int64  
 15  ECON            18395 non-null  int64  
 16  ENER            18395 non-null  int64  
 17  ENGI            18395 non-null 